In [1]:
import graphlab

In [3]:
products = graphlab.SFrame('amazon_baby.gl/')
products['word_count'] = graphlab.text_analytics.count_words(products['review'])
products = products[products['rating'] != 3]
products['sentiment'] = products['rating'] >= 4

In [5]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'bad', 'terrible', 'awful', 
                 'wow', 'hate']

In [6]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda x: x.get(word, 0))

In [7]:
for word in selected_words:
    print '{} {}'.format(word, products[word].sum())

awesome 2002
great 42420
fantastic 873
amazing 1305
bad 3197
terrible 673
awful 345
wow 131
hate 1057


prob1

In [8]:
train_data, test_data = products.random_split(.8, seed=0)

In [9]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                          target='sentiment',
                                                          features=selected_words,
                                                          validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 9

...

| 3         | 4        | 1.910980     | 0.842943          | 0.841400            |

| 4         | 5        | 2.124737     | 0.842943          | 0.841400            |

| 5         | 6        | 2.265211     | 0.842943          | 0.841400            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

prob2

In [14]:
selected_words_model['coefficients'].sort('value', ascending=False)

name,index,class,value,stderr
(intercept),None,1,1.54280952781,0.00823695499695
awesome,None,1,1.11924387519,0.109059823486
amazing,None,1,0.962148897981,0.126634020507
fantastic,None,1,0.929281631985,0.152851597693
great,None,1,0.902207530723,0.0215156608677
wow,None,1,-0.0883642137815,0.26636425354
bad,None,1,-0.982897555622,0.0418566929892
hate,None,1,-1.18870734644,0.0719426089118
awful,None,1,-1.75467363482,0.126778219489
terrible,None,1,-2.06763420341,0.093348737461


In [15]:
selected_words_model.evaluate(test_data)['accuracy']

0.8414004323804949

In [12]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 5.494357     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 8.833509     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [16]:
sentiment_model.evaluate(test_data)['accuracy']

0.916256305548883

In [27]:
temp = 1.0 * sum(products['sentiment']) / products.num_rows()
majority_class_accuracy = max(temp, 1 - temp)
majority_class_accuracy

0.8411233448474381

In [28]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ'].sort('rating', ascending=False)

In [33]:
sentiment_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.9999606821195993]

In [34]:
selected_words_model.predict(diaper_champ_reviews[:1], output_type='probability')

dtype: float
Rows: 1
[0.823872776784962]

In [22]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.823872776784962]